# This notebook is just my rough draft of text-based music generation. The formal version is in the notebook ABC.ipynb which documents a more detailed explanation of RNN & LSTM (preliminary knowledge).

In [4]:
import numpy as np 
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Dropout, CuDNNLSTM
from keras import optimizers
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback
import sys
import io

In [5]:
data = open('Nottingham.txt', 'r').read() # should be simple plain text file
chars = list(set(data)) #vocab of the training set
data_size, vocab_size = len(data), len(chars)
print ('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } #encoder
ix_to_char = { i:ch for i,ch in enumerate(chars) } #decoder
encoded_data = [char_to_ix[char] for char in data]

data has 435382 characters, 93 unique.


In [6]:
hidden_size = 512
seq_len = 100
learning_rate = 2e-3

In [7]:
dataX = [encoded_data[i:i+seq_len] for i in range(0, len(encoded_data)-seq_len-1, seq_len)] #input sequences
dataY = [encoded_data[i:i+seq_len] for i in range(1, len(encoded_data)-seq_len, seq_len)] #target sequences, exactly
#1 character away from the input sequences. Eg: input 'hell' expect to output 'ello'

In [8]:
def one_hot(vector, size): #I naively coded a one_hot function by myself. But I learned 
    #easier ways to do this later. In my other notebooks, I tried different methods.
    '''input a vector of vectorized sequences and output the one-hot version of the input vector'''
    one_hot = []
    for value in vector:
        x = [0 for _ in range(size)]
        x[int(value)] = 1
        one_hot.append(x)
    return one_hot

In [9]:
X = np.array([one_hot(example, vocab_size) for example in dataX])
Y = np.array([one_hot(example, vocab_size) for example in dataY])
print(f'X-shape: {X.shape}, Y-shape: {Y.shape}')

X-shape: (4353, 100, 93), Y-shape: (4353, 100, 93)


In [10]:
dataset = [(inp, targ) for inp, targ in zip(X, Y)]
random.shuffle(dataset) # I don't know why I decided to shuffle it. It turns out that when you re-run the notebook
#X, Y will be different. In this case, even when I save the model, I can not resume the training process but to 
#start over... VERY INJUDICIOUS CHOICE

In [11]:
X = np.array([pair[0] for pair in dataset])
Y = np.array([pair[1] for pair in dataset])

In [50]:
model = Sequential() #Two LSTM layers sandwiching dropouts architecture

model.add(LSTM(hidden_size, input_shape=(seq_len, vocab_size), return_sequences=True, activation='sigmoid')) 
# sigmoid activation is commonly used for binary categorical data (especially our one-hot input).
# set 'return_sequences=True' to match the dimension of my target output.
model.add(Dropout(0.5)) #to avoid overfitting

model.add(LSTM(hidden_size, input_shape=(seq_len, vocab_size), return_sequences=True, activation='sigmoid' ))
model.add(Dropout(0.5))

model.add(Dense(vocab_size, activation = 'softmax')) #Softmax will get me a probability output distribution 
#of all vocab. For example, let's assume our vocab list is the 26 alphabet in lower case. After I input 'h', 
#'e', 'l', 'l' in each timestep, the final output should give me a list of 26 probabilities
#each corresponding to an alphabet, which might look like [0.001,0.003...]. The probability that corresponds to
# the letter 'o', will presumably have the highest value if the model is well-trained so that we can get the 
# correct prediction.

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# 'adam' is just a version of stochastic gradient descent.

'Categorical_crossentropy' is a loss measurement for categorical data. It's often used together with "softmax".
It's also called the negative likelihood loss. The formula is $$loss = -\sum_{j=1}^{M}y_{j}\log{\hat{y_{j}}}$$

In [51]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100, 512)          1241088   
_________________________________________________________________
dropout_5 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_6 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 100, 93)           47709     
Total params: 3,387,997
Trainable params: 3,387,997
Non-trainable params: 0
_________________________________________________________________


In [181]:
def generate_music(model,seq_len,vocab_size):
    result = []
    start_index = random.randint(0, len(data) - seq_len - 1)
    sentence = data[start_index: start_index + seq_len]
    encoded_sentence = [char_to_ix[char] for char in sentence]
    for i in range(5):
        inp = np.array(one_hot(encoded_sentence, vocab_size)).reshape(1,seq_len,vocab_size)
        out = model.predict_classes(inp)[0] #100 predicted values 
        for value in out:
            result.append(value)
        #result.append(out[-1])
        encoded_sentence = out # use these 100 values to predict another 100 values
    decoded_result = ''
    for value in result:
        decoded_result += ix_to_char[value]
    return decoded_result #a decoded result with len=500
        
        

The model was trained super slow and not performed very well. I did not clean the data and my method of generating the music was certainly problematic. Since most idealistically, if the model had 100% accuracy, by the music generation function I coded, it would predict the same 99 overlapping notes again. Using the hello analogy, it would give me something like "helloellox", which is not what I meant to do. I decided to do a more polished version. You can view the updated one in ABC.ipynb

In [131]:
model.fit(X,Y, epochs= 100)

Epoch 1/100
4353/4353 [==============================] - 450s 103ms/step - loss: 1.5264 - accuracy: 0.5497
Epoch 2/100
4353/4353 [==============================] - 424s 97ms/step - loss: 1.4930 - accuracy: 0.5560
Epoch 3/100
4353/4353 [==============================] - 437s 100ms/step - loss: 1.4609 - accuracy: 0.5623
Epoch 4/100
4353/4353 [==============================] - 424s 97ms/step - loss: 1.4368 - accuracy: 0.5670
Epoch 5/100
4353/4353 [==============================] - 424s 97ms/step - loss: 1.4161 - accuracy: 0.5717
Epoch 6/100
4353/4353 [==============================] - 418s 96ms/step - loss: 1.3970 - accuracy: 0.5758
Epoch 7/100
4353/4353 [==============================] - 418s 96ms/step - loss: 1.3757 - accuracy: 0.5800
Epoch 8/100
4353/4353 [==============================] - 433s 100ms/step - loss: 1.3600 - accuracy: 0.5823
Epoch 9/100
4353/4353 [==============================] - 453s 104ms/step - loss: 1.3452 - accuracy: 0.5863
Epoch 10/100
4353/4353 [==================

KeyboardInterrupt: 

In [132]:
generate_music(model,seq_len,vocab_size) #result after 20 epochs

'G"c2 |3e/||"D""A2|\n\n3"D"c2c  ]"3"]]]]]"3"D""A2d  ]"3e]]]]]"D""d2AG22G2A|]"D"e2c2]"2c|\n\n\nX: 11\nT:Tont/G2B""\n/2\n\nD"dA""""\n/A"d/|/"""D"e""||\nA\nA"AD7"|"""D"/|[|||A7dA"||]]]|]|""D"A2d ][D"d""\nX: 11\nT:The e2 |/A"":A"\n2A2]AA""":AA22"3AG""d/AA"\n\n/"/A7""A"DD""A2"1"\n\n\n"/]A"\n\n[|\n\n\nDD"d2 |""c\nG2cA": 1\n\nT:The Br|""2BGD"/A": ]|]]]G""|]]]A\n]]GD"2F/A"\n2G2d"AD7A2FDD7"D"D"\nXD""2A"\n3"\n\nX\n\nG2|"\nDD""/||]A"[\n"\n:The Bre"DD"/  G2FD"\n:\n\n\n||GG"\n:|2|:A||]||2FD"/ ||A7|||]|]||G2D2D":2GD"|G"|D"\n: \n2|\nD"\n"GD2"\n\n2A""D""2e\nB2e\n'

In [133]:
model.fit(X,Y, epochs= 100)

Epoch 1/100
4353/4353 [==============================] - 478s 110ms/step - loss: 1.3047 - accuracy: 0.5956
Epoch 2/100
4353/4353 [==============================] - 463s 106ms/step - loss: 1.2838 - accuracy: 0.5998
Epoch 3/100
4353/4353 [==============================] - 443s 102ms/step - loss: 1.2758 - accuracy: 0.6022
Epoch 4/100
4353/4353 [==============================] - 445s 102ms/step - loss: 1.2643 - accuracy: 0.6044
Epoch 5/100
4353/4353 [==============================] - 449s 103ms/step - loss: 1.2558 - accuracy: 0.6060
Epoch 6/100
4353/4353 [==============================] - 444s 102ms/step - loss: 1.2456 - accuracy: 0.6087
Epoch 7/100
4353/4353 [==============================] - 439s 101ms/step - loss: 1.2408 - accuracy: 0.6097
Epoch 8/100
4353/4353 [==============================] - 446s 102ms/step - loss: 1.2302 - accuracy: 0.6134
Epoch 9/100
4353/4353 [==============================] - 443s 102ms/step - loss: 1.2214 - accuracy: 0.6153
Epoch 10/100
4353/4353 [=============

KeyboardInterrupt: 

In [134]:
model.save('model.h5')

In [139]:
generate_music(model,seq_len,vocab_size) #result after 86 epochs

'|"/2d/2g/2e/2|"D"d/ B/2c/2B/2d/2|"Am"c/2B/2e/2B/2 "D7A/2G/2F/2A/2|\\\n"G"G/2B/2d/2c/2 "D7"B/2A/2G/2A/2"D@"/2e/2f/2f"D"f/2d/4A/2d/2 /2 "A""c/2d/2A/2c/2 "D7"/fc/2F/2G/2|\n\n"G"G/2B/2G/2B/2 BD7"B/2A/2G/2A/2|A"tA@A/2f/2g/A"f/2e/2|/4 /2|"2d"D7cA#4+/2e/2|/2|"D""d2//2d/2 /2|""\nG"G/2B/2G/2B/2 B///G/2G/2G/2B/2|"/Aa  \n2B/4|/2|A#4a/2|"2f"4f"D"/f""///||2|/2|"2f"D"ff""22f/2 f2 "DD""e/2f/2 /2A/2 "/2cc/2z/2|/2|/2|"C2|/""" 22c"4""//d/4g"A"/a"/a/f2 eb@)a""""2""D"/g"f//A"" /2|"/|"C7"ff"f"/2g"2"/2|"G@"//2|/2|\n2 "2""D"'

In [140]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 471s 108ms/step - loss: 0.9475 - accuracy: 0.6939
Epoch 2/100
4353/4353 [==============================] - 466s 107ms/step - loss: 0.9491 - accuracy: 0.6931
Epoch 3/100
4353/4353 [==============================] - 469s 108ms/step - loss: 0.9437 - accuracy: 0.6951
Epoch 4/100
4353/4353 [==============================] - 462s 106ms/step - loss: 0.9439 - accuracy: 0.6954
Epoch 5/100
4353/4353 [==============================] - 462s 106ms/step - loss: 0.9397 - accuracy: 0.6969
Epoch 6/100
4353/4353 [==============================] - 474s 109ms/step - loss: 0.9353 - accuracy: 0.6969
Epoch 7/100
4353/4353 [==============================] - 490s 113ms/step - loss: 0.9339 - accuracy: 0.6977
Epoch 8/100


KeyboardInterrupt: 

In [141]:
model.save('model.h5')  #94 epoch

In [143]:
model.fit(X,Y, epochs = 10)

Epoch 1/10
4353/4353 [==============================] - 374s 86ms/step - loss: 0.9308 - accuracy: 0.6990
Epoch 2/10
4353/4353 [==============================] - 371s 85ms/step - loss: 0.9286 - accuracy: 0.6995
Epoch 3/10
  64/4353 [..............................] - ETA: 6:26 - loss: 0.9057 - accuracy: 0.7045

KeyboardInterrupt: 

In [144]:
model.save('model.h5') #97 epochs

In [145]:
model.fit(X,Y, epochs = 10)

Epoch 1/10
4353/4353 [==============================] - 376s 86ms/step - loss: 0.9268 - accuracy: 0.6995
Epoch 2/10
4353/4353 [==============================] - 381s 87ms/step - loss: 0.9207 - accuracy: 0.7019
Epoch 3/10
4353/4353 [==============================] - 387s 89ms/step - loss: 0.9218 - accuracy: 0.7011
Epoch 4/10
4353/4353 [==============================] - 399s 92ms/step - loss: 0.9180 - accuracy: 0.7027
Epoch 5/10
4353/4353 [==============================] - 374s 86ms/step - loss: 0.9160 - accuracy: 0.7036
Epoch 6/10
4353/4353 [==============================] - 372s 86ms/step - loss: 0.9127 - accuracy: 0.7042
Epoch 7/10
1088/4353 [======>.......................] - ETA: 10:00 - loss: 0.9100 - accuracy: 0.7043

KeyboardInterrupt: 

In [146]:
model.save('model.h5') #103 epochs

In [164]:
generate_music(model,seq_len,vocab_size)

'|/2|"/||""D""D/ F/|2G/2|"D"A/2B/2F/2G/2 A/2G/2F/2E/2|"D7"F/2B/2F/"D7"E/|"Gm"D/ D:|\n"/"Gm"f/ d/|"Gm"c"2 "G@"\nDD"ff"f"/2"D/2F"D"F/2F/2 /2A/2 "/2F/2F/2A/2|\\D""D/2G/2A/2G/"F/2FD""D/2F/|\n\nC@A//G/2d/2fG""g/A""D""G"""G2|e#d2 f7f+/f"f|2F/2|"2F/2FFF@"/2A/2D/2F"\n/DG"f+/2F/2F/2 /2 // D"f+/2""\nF/22F/2F/2c//|D"b/AA"AA"BA""""2" |"2"||2+e/"" 2A"G"/2F// 2C@E/2F/2F/A":C/E/"2c/2|/2GF2E+22c/A/"cceb"/2A+/2[/2A/22"/F"2|/A"AA/|AA"""A""A"E"""322cB""/e"B2 /22"+//|2|/2|/2|D"\n2 2C"/2 "2F//]/[||/2|2F//||A2c/|2Ec2c/2A|A2AA'

In [165]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 492s 113ms/step - loss: 0.9090 - accuracy: 0.7060
Epoch 2/100
4353/4353 [==============================] - 435s 100ms/step - loss: 0.9062 - accuracy: 0.7065
Epoch 3/100
4353/4353 [==============================] - 449s 103ms/step - loss: 0.9061 - accuracy: 0.7061
Epoch 4/100
4353/4353 [==============================] - 455s 104ms/step - loss: 0.9021 - accuracy: 0.7071
Epoch 5/100
4353/4353 [==============================] - 498s 114ms/step - loss: 0.8989 - accuracy: 0.7085
Epoch 6/100
4353/4353 [==============================] - 474s 109ms/step - loss: 0.8986 - accuracy: 0.7080
Epoch 7/100
4353/4353 [==============================] - 475s 109ms/step - loss: 0.8955 - accuracy: 0.7089
Epoch 8/100
4353/4353 [==============================] - 546s 125ms/step - loss: 0.8968 - accuracy: 0.7083
Epoch 9/100
4353/4353 [==============================] - 405s 93ms/step - loss: 0.8904 - accuracy: 0.7106
Epoch 10/100
4353/4353 [==============

KeyboardInterrupt: 

In [166]:
model.save('model.h5') #120 epochs

In [167]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 387s 89ms/step - loss: 0.8709 - accuracy: 0.7165
Epoch 2/100
4353/4353 [==============================] - 435s 100ms/step - loss: 0.8677 - accuracy: 0.7171
Epoch 3/100
4353/4353 [==============================] - 468s 107ms/step - loss: 0.8677 - accuracy: 0.7173
Epoch 4/100
4353/4353 [==============================] - 382s 88ms/step - loss: 0.8630 - accuracy: 0.7191
Epoch 5/100
4353/4353 [==============================] - 2995s 688ms/step - loss: 0.8645 - accuracy: 0.7178
Epoch 6/100
4353/4353 [==============================] - 1520s 349ms/step - loss: 0.8590 - accuracy: 0.7197
Epoch 7/100
4353/4353 [==============================] - 1505s 346ms/step - loss: 0.8595 - accuracy: 0.7197
Epoch 8/100
4353/4353 [==============================] - 1463s 336ms/step - loss: 0.8553 - accuracy: 0.7209
Epoch 9/100
4353/4353 [==============================] - 3282s 754ms/step - loss: 0.8508 - accuracy: 0.7219
Epoch 10/100
4353/4353 [==========

KeyboardInterrupt: 

In [168]:
model.save('model.h5') #131 epochs

In [169]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 377s 87ms/step - loss: 0.8430 - accuracy: 0.7247
Epoch 2/100
4353/4353 [==============================] - 370s 85ms/step - loss: 0.8438 - accuracy: 0.7246
Epoch 3/100
4353/4353 [==============================] - 670s 154ms/step - loss: 0.8422 - accuracy: 0.7243
Epoch 4/100
4353/4353 [==============================] - 384s 88ms/step - loss: 0.8410 - accuracy: 0.7254
Epoch 5/100
4353/4353 [==============================] - 380s 87ms/step - loss: 0.8390 - accuracy: 0.7255
Epoch 6/100
4353/4353 [==============================] - 365s 84ms/step - loss: 0.8343 - accuracy: 0.7262
Epoch 7/100
4353/4353 [==============================] - 362s 83ms/step - loss: 0.8294 - accuracy: 0.7284
Epoch 8/100
4353/4353 [==============================] - 363s 83ms/step - loss: 0.8315 - accuracy: 0.7279
Epoch 9/100
4353/4353 [==============================] - 362s 83ms/step - loss: 0.8282 - accuracy: 0.7292
Epoch 10/100
4353/4353 [=====================

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3325, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-169-24da9b7717fa>", line 1, in <module>
    model.fit(X,Y, epochs = 100)
  File "//anaconda3/lib/python3.7/site-packages/keras/engine/training.py", line 1239, in fit
    validation_freq=validation_freq)
  File "//anaconda3/lib/python3.7/site-packages/keras/engine/training_arrays.py", line 196, in fit_loop
    outs = fit_function(ins_batch)
  File "//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py", line 3740, in __call__
    outputs = self._graph_fn(*converted_inputs)
  File "//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py", line 1081, in __call__
    return self._call_impl(args, kwargs)
  File "//anaconda3/lib/python3.7/site-packages/tensorflow_core/python/eager/function.py", line 1121, in _call_impl
    retu

KeyboardInterrupt: 

In [170]:
model.save('model.h5') #199 epochs

In [171]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 3050s 701ms/step - loss: 0.7154 - accuracy: 0.7623
Epoch 2/100
4353/4353 [==============================] - 3886s 893ms/step - loss: 0.7128 - accuracy: 0.7629
Epoch 3/100
4353/4353 [==============================] - 453s 104ms/step - loss: 0.7156 - accuracy: 0.7626
Epoch 4/100
4353/4353 [==============================] - 390s 90ms/step - loss: 0.7161 - accuracy: 0.7623
Epoch 5/100
4353/4353 [==============================] - 414s 95ms/step - loss: 0.7090 - accuracy: 0.7642
Epoch 6/100
4353/4353 [==============================] - 373s 86ms/step - loss: 0.7065 - accuracy: 0.7662
Epoch 7/100
4353/4353 [==============================] - 378s 87ms/step - loss: 0.7076 - accuracy: 0.7645
Epoch 8/100
4353/4353 [==============================] - 388s 89ms/step - loss: 0.7039 - accuracy: 0.7664
Epoch 9/100
4353/4353 [==============================] - 371s 85ms/step - loss: 0.7013 - accuracy: 0.7666
Epoch 10/100
4353/4353 [=================

KeyboardInterrupt: 

In [186]:
model.save('model.h5') #206

In [ ]:
model.fit(X,Y, epochs = 100)

Epoch 1/100
4353/4353 [==============================] - 391s 90ms/step - loss: 0.6875 - accuracy: 0.7714
Epoch 2/100
4353/4353 [==============================] - 382s 88ms/step - loss: 0.6926 - accuracy: 0.7694
Epoch 3/100
4353/4353 [==============================] - 386s 89ms/step - loss: 0.6882 - accuracy: 0.7713
Epoch 4/100
4353/4353 [==============================] - 377s 87ms/step - loss: 0.6886 - accuracy: 0.7713
Epoch 5/100
4353/4353 [==============================] - 369s 85ms/step - loss: 0.6807 - accuracy: 0.7732
Epoch 6/100
4353/4353 [==============================] - 372s 85ms/step - loss: 0.6823 - accuracy: 0.7725
Epoch 7/100
4353/4353 [==============================] - 373s 86ms/step - loss: 0.6828 - accuracy: 0.7728
Epoch 8/100
4353/4353 [==============================] - 369s 85ms/step - loss: 0.6805 - accuracy: 0.7732
Epoch 9/100
4353/4353 [==============================] - 371s 85ms/step - loss: 0.6772 - accuracy: 0.7740
Epoch 10/100
4353/4353 [======================

In [ ]:
model = load_model('model.h5')